In [1]:
%%HTML
<style> code {background-color : lightgrey !important;} </style>

# About this document

The purpose of this noted version of pipeline is to guide the user through each step of the codes so that they are able to understand them as much as possible and equipped with knowledge to customize the script regardless of programming skills. It is **not** supposed to be runned as production tool. If you click "Run All" on this document, you **will** encounter errors. For productive purpose, consider modifying the accompanying **pipeline.ipynb** to suit your need.

Before we start, it's highly recommended that you get familiar with basic python concepts and operations like [string manipulation](https://docs.python.org/3.4/library/string.html), [tuples, lists and dictionaries](https://docs.python.org/3/tutorial/datastructures.html), as well as a little bit about [object-oriented programming](https://python.swaroopch.com/oop.html) and [python modules](https://docs.python.org/3/tutorial/modules.html).

Another note on the styling of this document: most of the sentences should (hopefully) make sense if taken literally. However, I try to use some syntax highlighting on the texts consistently to demonstrate the close relationship between the codes and thought process, as well as encouraging the reader to understand the already natural and self-explantory Python syntax and jargons. Specifically:

-  a [hyperlink](https://en.wikipedia.org/wiki/Hyperlink) usually point to a well-defined python module or class or methods, especially when that concept is first encountered in this document and the reader would likely need some background reference. The link usually points to the official documentation of that concept, which might not be the best place to start for beginner. If you find the documentation puzzling, try to google that concept and find a tutorial that best suit your preference.
-  an inline `code` usually refer to a name that already exsist in the [namespace](https://docs.python.org/3/tutorial/classes.html#python-scopes-and-namespaces) (i.e. the context where we run the codes in this document). It can be a previously encountered concepts, but more often it referes to variable or method names that we [imported](https://docs.python.org/3/reference/import.html) or defined along the way.
-  **bold** texts are used more loosely to highlight anything that requires more attention than plain texts. Though it's not used as carefully as previous formats, it often refer to some specific values that a variable or method arguments can assume.

# Pipeline

## Setting up

The cells under this section should be executed every time the kernel is restarted.

### set module paths and data path

Ideally, the following cell would be the only thing you have to change when analyzing different datasets. `minian_path` should be the path that contains a folder named **"minian"** , under which the actual codes of **minian** (.py files) resides. The default value **\.** means "current folder", which should work in most cases unless you want to try out another specific version of minian that is not in the same folder as this notebook. `dpath` is the folder that contains the actual videos, which are usually named **"msCam\*.avi"** where \* is a number. `meta_dict` is a python [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) that is used to construct meta data for the final labeled data structure. Currently we assume data are stored in some structured folders and we use folder names to fill in information. The default value means: the name of the last folder in `dpath` (the one that directly contains the videos) would be used as the value of a field named **'session_id'**, the name of the second-last folder in `dpath` would be the value for **'session'** and so on. Both the `keys` (field names) and `values` (numbers indicating which level of folder name should be used) of `meta_dict` can be modified to suit the specific user case. `chunks` is used to divide data into smaller chunks for parallel processing. Currently it is rarelly used and has little impact on the actual performance of the script. It is recommended to leave `chunks` to default. `in_memory` specify whether to load the whole dataset into memory. Since out-of-core computation are still not fully supported, it is recommended to leave `in_memory` to `True`.

In [ ]:
minian_path = "."
dpath = "./demo_movies/"
meta_dict={'session_id': -1, 'session': -2, 'animal': -3}
chunks = {'frame': 1000, 'height': 200, 'width': 200, 'unit_id':20}
in_memory = True

### load modules

The following cell loads the necessary modules and usually should not be modified. If you encounter an `ImportError`, check that you followed the installation instructions and that `minian_path` is pointing to the right place.

In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(minian_path)
import gc
import psutil
import numpy as np
import xarray as xr
import holoviews as hv
import paramnb
import matplotlib.pyplot as plt
import bokeh.plotting as bpl
import dask.array as da
import pandas as pd
import dask
import datashader as ds
from holoviews.operation.datashader import datashade, regrid, dynspread
from datashader.colors import Sets1to3
from dask.diagnostics import ProgressBar
from IPython.core.display import display, HTML
from dask.distributed import Client, progress
from minian.utilities import load_videos, varray_to_tif, save_cnmf, save_movies, scale_varr, save_variable
from minian.preprocessing import remove_brightspot, gradient_norm, denoise, remove_background, stripe_correction
from minian.motion_correction import estimate_shift_fft, apply_shifts, interpolate_frame, mask_shifts
from minian.initialization import seeds_init, gmm_refine, pnr_refine, intensity_refine, ks_refine, seeds_merge, initialize
from minian.cnmf import get_noise_fft, get_noise_welch, update_spatial, update_temporal, unit_merge
from minian.visualization import VArrayViewer, MCViewer, CNMFViewer, generate_videos, visualize_temporal_update, normalize

### module initialization

normalize `dpath` and initialize package for visualization. Re-run this cell if you are having trouble getting the visualization to show up on the notebook.

In [ ]:
dpath = os.path.abspath(dpath)
hv.notebook_extension('bokeh', width=100)

## Pre-processing

### loading videos and visualization

The first argument of `load_videos` should be the path that contains the videos, which could be `dpath` we already defined. The second argument `pattern` is optional and is the [regular expression](https://docs.python.org/3/library/re.html) used to filter files under the specified folder. The default value **'msCam[0-9]+\.avi$'** means that a file can only be loaded if its filename contain **'msCam'** then followed by at least one digit of number then **'.avi'** as the end of the filename. This can be changed to suit the naming convention of your videos. `in_memory` specify whether the whole video should be loaded into memory. Currently it is recommended to set `in_memory` to `True`. `dtype` is the underlying [data types](https://docs.scipy.org/doc/numpy-1.15.0/user/basics.types.html) of the data. Usually `float32` is good enough and should be preferred to save memory demand. The resulting "video array" `varr` contains three dimensions: `height`, `width`, and `frame`. If you wish to downsample the video, pass in a `dictionary` to `resample`, whose keys should be the name of dimensions and values an integer specifying how many times that dimension should be reduced. For example, `resample={'frame': 2}` will temporally downsample the video with a factor of 2.

In [ ]:
%%time
varr = load_videos(dpath, in_memory=in_memory, dtype=np.float32, resample=None)

The previous cell load videos and concatenate them together into `varr`, which is a [xarray.DataArray](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.html#xarray.DataArray). Now it's a perfect time to get familiar with this data structure and the [xarray](https://xarray.pydata.org/en/stable/) module in general, since we will be using these data structures throughout analysis. Basicly a `xarray.DataArray` is a labeled N-dimensional array. We can ask the computer to print out some information of `varr` by calling its name (as with any other variable):

In [ ]:
varr

We can see now that `varr` is a `xarray.DataArray` with a [name](https://xarray.pydata.org/en/stable/generated/xarray.DataArray.name.html#xarray.DataArray.name) `'demo_movies'` and three dimensions: `frame`, `height` and `width`, each dimension is simply labeled with ascending natural numbers. The [dtype](https://xarray.pydata.org/en/stable/generated/xarray.DataArray.dtype.html#xarray.DataArray.dtype) ([data type](https://docs.scipy.org/doc/numpy-1.14.0/user/basics.types.html)) of `varr` is `numpy.float32`

In addition to these information, we can visualize `varr` with the help of `VArrayViewer`, which shows the array as movie and also plot max, mean and minimum fluorescence:

In [ ]:
%%output size=100
vaviewer = VArrayViewer([varr], framerate=5)
display(vaviewer.widgets)
vaviewer.show()

### subset part of video

Before proceeding to pre-processing, it's good practice to check if there is something obviously wrong with the video (like camera suddenly drops dark). This can usually be observed by visualizing the video and check the mean fluorescence plot. To take out some bad `frame`, let's say, `frame` after 800, we can utilize the [xarray.DataArray.sel](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.sel.html) method and [slice](https://docs.python.org/3/library/functions.html#slice):

In [ ]:
varr_ref = varr.sel(frame=slice(None, 800))

This will subset `varr` along the `frame` dimension from the begining to the `frame` labeled **800**, and assign the result back to `varr_ref`, which is equivalent to taking out `frame` from **801** to the end. Note you can do the same thing to other dimensions like `height` and `width` to take out certain pixels of your video for all the `frame`s. For more information on using `xarray.DataArray.sel` as well as other indexing strategies, see [xarray documentation](http://xarray.pydata.org/en/stable/indexing.html)

If your `varr` is fine, just assign it to `varr_ref` to keep the naming consitent with later codes.

In [ ]:
varr_ref = varr

Now we chunk `varr` into smaller pieces to use parallelization to accelerate performance.

In [ ]:
varr_ref = varr_ref.chunk(dict(height=-1, width=-1, frame='auto'))

### bright spots removal

`remove_brightspot` tries to correct for brightspots that are probably caused by dust on the imaging sensor. Usually this type of artifacts occupies one pixel which is significantly brighter than its surrounding pixels. Thus, for any given pixel, `remove_brightspot` calculate a difference of intensity between that pixel and the mean of the **four** surrounding pixels (that is, four pixels that are connected to the pixel in question by edge). Then for each frame, this difference value is taken for all pixels, and a zscore of them is calculated. The pixels whose zscore of difference that exceed `thres` is treated as "brightspots", and whose value will be substituted by the mean of its four surrounding pixels. Thus `thres` controls how strict brightspot detection would be. In stead of passing in numbers, you can also pass a string `'min'`, which means use the inverse of the minimum (most negative) value of zscore as the threshold. If you believe your videos are free of this type of artifacts, feel free to skip this step to reduce artificial assumptions and biases.

For the following few cells during pre-processing, the functions (such as `remove_brightspot`) are evaluated lazily, which means it only creates a "plan" for the comuptation without doing it. Actual computations are carried out when we explicitly call the [persist](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.persist.html) method of the resulting `DataArray` (like `varr_ref.persist()`). This opens up possibility for [out-of-core computations](https://en.wikipedia.org/wiki/External_memory_algorithm), but it's currently not tested yet. So for now it is recommended to leave everything under the `if in_memory:` clause intact.

In [ ]:
%%time
varr_ref = remove_brightspot(varr_ref, thres=2)
if in_memory:
    with ProgressBar():
        varr_ref = varr_ref.persist()

### stripe correction

The earlier versions of miniscopes suffer from artifacts that looks like horizontal or vertical stripes across the image that's stationary across frames. `stripe_correction` tries to correct for this type of artifacts. Technically it calculate the mean along `'frame'` dimension (a "mean frame" if you will), then again calculate a mean along `reduce_dim` to get a one-dimensional representation of the "stripes", and then every frame in the movie is subtracted by this 1d stripe uniformly. Thus, `reduce_dim` should be the dimension (direction) the stripes run along. If you believe your videos are free from the stripes artifacts, feel free to skip this step to reduce artificial assumptions and biases.

In [ ]:
%%time
varr_ref = stripe_correction(varr_ref, reduce_dim='height')
if in_memory:
    with ProgressBar():
        varr_ref = varr_ref.persist()

### normalize

`scale_varr` is a convenient function that act on `DataArray`s to linearly rescale every value to the range of 0 to 1 inclusive. If you don't like it, pass in `scale` to change the range or skip it altogether. Note that `scale` takes a `tuple` of 2 elements representing lower and upper bound of the range.

In [ ]:
%%time
varr_ref = scale_varr(varr_ref, scale=(0, 1))
if in_memory:
    with ProgressBar():
        varr_ref = varr_ref.persist()

### save processed movie

The `save_variable` function takes in three required arguments: `var` is the variable you want to save, `fpath` is the path under which the actual data file will be stored, and `fname` is the file name of the data file. `meta_dict` is optional and will be used to fill in metadatas.

In particular, here we are saving our minimally-processed videos in `DataArray` format. We give it a "name" `"org"` by calling the [rename](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.rename.html) method on the array, which is `xarray`'s internal naming system that stays with the actual data and will be displayed when you print out the `DataArray` (In contrast, you can bind arbitrary variable name to the data like `whatever = varr_ref` without touching the data, which is less reliable and not accessible to other functions, which is probably why we need another name). In practice, give it a name that's human-readable and be sure to not name two pieces of data with the same name (Otherwise an error will occur if you try to combine them in a single dataset). In any case, the saved original videos are usually not used in the later part of the pipeline, except for the final `generate_videos` method which generate a movie to visualize the result of the pipeline. Thus if you are sure that you don't care about that movie, feel free to skip this to save both time and disk space.

In [ ]:
%%time
save_variable(var=varr_ref.rename("org"), fpath=dpath, fname='minian', meta_dict=meta_dict)

### estimate gradient

The anisotropic diffusion in the next step requires a parameter `kappa`, which is best estimated with the [gradient](https://en.wikipedia.org/wiki/Image_gradient) of the image. See the description of the next step for details about this parameter and the whole process. Technically, here we choose the 90-th percentile `varr_gradient.quantile(0.9)` of the gradient of the first frame `varr_ref.isel(frame=0)` as kappa. If you believe this is a bad choice, or not want to use anisotropic diffusion at all, feel free to change or skip this step.

In [ ]:
%%time
with ProgressBar():
    varr_gradient = gradient_norm(varr_ref.isel(frame=0)).compute()

In [ ]:
kappa = varr_gradient.quantile(0.9).values

### denoising

This step carry out denoising of the image frame by frame. The method `denoise` takes in two required arguments: the first is the video array `varr`, and the second `method` is a string specifying the denoising method we are gonna use. Right now two methods are supported: `'gaussian'` and `'anisotropic'`. Under the hood, `denoise` simply call another function frame by frame in a parallel fashion. For `method='gaussian'` it calls [GaussianBlur](https://www.docs.opencv.org/3.3.0/d4/d86/group__imgproc__filter.html#gaabe8c836e97159a9193fb0b11ac52cf1) from `OpenCV` package. For `method='anisotropic'` it calls [anisotropic_diffusion](http://loli.github.io/medpy/generated/medpy.filter.smoothing.anisotropic_diffusion.html) from `medpy` package. All additional [keyword arguments](https://docs.python.org/3.7/tutorial/controlflow.html#keyword-arguments) passed into `denoise` are directly passed into one of those two denoising functions under the hood.

[Gaussian blur](https://en.wikipedia.org/wiki/Gaussian_blur) is straight-forward. All you need to do is pass in either the size of the gaussian kernel `ksize`, or the standard deviation of the kernel `sigmaX` and `sigmaY`. Pragmatically, `ksize=(3, 3)` with `sigmaX=0` (meaning adjust the standard deviation according to kernel size) works fairly well. For full controll of the process, refer to the [documentation](https://www.docs.opencv.org/3.3.0/d4/d86/group__imgproc__filter.html#gaabe8c836e97159a9193fb0b11ac52cf1).

In [ ]:
Y = denoise(varr=varr_ref, method='gaussian', ksize=(3, 3), sigmaX=0) 

Using [anisotropic diffusion](https://en.wikipedia.org/wiki/Anisotropic_diffusion) is inspired by [MIN1PIPE](https://github.com/JinghaoLu/MIN1PIPE) package. Refer to [the paper](https://www.cell.com/cell-reports/fulltext/S2211-1247(18)30826-X) for full detail. In a nutshell, gaussian blur can be think of as a diffusion process - that is, imagine your grey-scale image represent the concentration of some molecule, say saline, in a two dimesional dish, letting the saline diffuse for some time would be equivalent to carrying out a gaussian blur on the original image/concentration, and longer diffusion time correspond to gaussian blur with larger standard deviation. Anisotropic diffusion is a generalization of this idea - instead of diffusing uniformly, it make the diffusion slower where the gradient is higher. The result is a diffusion/denoising process that preserve the edges, and in theory is more favorable than gaussian blur. That said, `niter` is the number of iteration of this process, in other words, the "time-step" the diffusion is gonna last. The larger this number, the more blurred the image is gonna be. Pragmatically `niter=10` is good enought. Note that increasing this number will significantly increase the computation time since there more iterative steps for every frame. `kappa` decides who gets treated as "edges" and thus preserved. Pixels whose gradient is much larger than this number will diffuse much slower, whereas pixels with gradient lower than `kappa` will just flow. As mentioned before, pragmatically we found estimating `kappa` with the 90-th percentile of the gradient of an example frame works well. `gamma` control the speed of diffusion. In theory we want this as large as possible to save some `niter` and thus computation time. However it is advised to set this no larger than **0.25** otherwise weird things might happen. There were two equations that were proposed to control the diffusion process in the [original paper](https://dl.acm.org/citation.cfm?id=78304) of anisotropic diffusion, and `option` controls which one to use. Pragmatically `option=2` works better. For full control of the process, refer to the [documentation](http://loli.github.io/medpy/generated/medpy.filter.smoothing.anisotropic_diffusion.html).

In [ ]:
Y = denoise(varr=varr_ref, method='anisotropic', niter=10, kappa=kappa, gamma=0.25, option=2)

Lastly, don't forget to actually run the computation.

In [ ]:
%%time
if in_memory:
    with ProgressBar(), dask.config.set(scheduler='processes'):
        Y = Y.persist()

### background removal

This step try to estimate backgrounds frame by frame and remove them. As with last step, the first argument to `remove_backgroun` is `varr` which is the video array to be processed, and the second is the `method` to use. Again there are two methods available: `'uniform'` or `'tophat'`. For both of them, they require a single parameter - a window size `wnd`, which is the third required argument to `remove_background`. The two methods differ in how background is estimated. For `method='uniform'`, a [uniform filter](https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.ndimage.uniform_filter.html) (basically a two dimensional rolling mean) is applied to each frame. `wnd` control the window size of the filter, and the result is used as the background. The idea is that backgrounds mainly contains out-of-focus fluorescence, that is basicly a super-blurred version of the image itself. Thus a large (comparing to the expected size of a cell) number of `wnd` should work well. Pragmatically **50** works pretty good. For `method='tophat'`, a [disk element](http://scikit-image.org/docs/dev/api/skimage.morphology.html#disk) with a radius of `wnd` is created, then a [morphological erosion](https://homepages.inf.ed.ac.uk/rbf/HIPR2/erode.htm) using the disk element is applied to each frame, which eats away any bright "features" that is smaller than the disk element. Then, a [morphological dilation](https://homepages.inf.ed.ac.uk/rbf/HIPR2/dilate.htm) is applied to the "eroded" image, which in theory undo the erosion except the bright "features" that are completely eaten away will not be recovered. The overall effect of this process is to remove any bright feature that is smaller than a disk with radius `wnd`. Thus, when setting `wnd` to the expected size of **largest** cell, this process can give us a good estimation of the background. Pragmatically `wnd=10` works pretty well. The computation time of `method='tophat'` is significantly longer than `method='uniform'`, but the result is also visually much more appealing. Lastly, regardless of which `method` is chosen, `remove_background` will subtract the estimated background from the original movie frame by frame.

In [ ]:
%%time
Y = remove_background(varr=Y, method='tophat', wnd=10)
if in_memory:
    with ProgressBar(), dask.config.set(scheduler='processes'):
        Y = Y.persist()

### normalization

Here again we rescale the video array to `range=(0, 1)`. Nothing is passed in here since we can just use the default of `scale_varr` (which, by the way, can be viewed with **Shift + Tab** when your text cursor is pointing to `scale_varr`, or any other function).

In [ ]:
%%time
Y = scale_varr(Y)
if in_memory:
    with ProgressBar():
        Y = Y.persist()

### visualization of pre-processing

Here we visualize the result of pre-processing. `VArrayViewer` can be used to visualize videos side by side when passed in a [list](https://docs.python.org/3/tutorial/introduction.html#lists) of video arrays.

In [ ]:
%%output size=70
vaviewer = VArrayViewer([varr_ref, Y.rename('Y')], framerate=5)
display(vaviewer.widgets)
vaviewer.show()

## motion correction

### prepare movie for motion correction

#### use old method to generate movie for motion correction

`method='tophat'` for background removal usually works pretty well at extracting only the neurons -- in fact it is a little bit too well for the purpose of motion correction, since we have to assume the fluorescence from the cells at adjacent frames are stationary otherwise there is no other landmarks we can use to perform motion correction. Sometimes this is a good enough assumption while other times this is not. In that case it is better to generate a video solely for the purpose of motion correction using `method='uniform'` for `remove_background`, which only remove the general blurred out-of-focus fluorescence, while preserving other details such as, most importantly, blood vessels. Note that we now have two copies of our videos: the minially processed original movie `varr_ref` and denoised, background-free `Y`. The cell below creates another one `varr_mc` for motion correction. This is very memory-demanding and you might easily run into `MemoryError`. A workaround (before we have full out-of-core computation support) is to dump `varr_ref` and `Y` into hard drive using `save_variable`, or directly [to_netcdf](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.to_netcdf.html) from `xarray`, then restart the kernel to clear out memories, load `varr_ref` using [open_dataarray](http://xarray.pydata.org/en/stable/generated/xarray.open_dataarray.html), get the `shifts` using the **estimate shifts** section below, and apply them to `Y`.

In [ ]:
%%time
varr_mc = remove_background(varr_ref, 'uniform', 51)
varr_mc = denoise(varr_mc, 'gaussian', ksize=(3, 3), sigmaX=0)
if in_memory:
    with ProgressBar():
        varr_mc = varr_mc.compute()

#### use Y for motion correction

Another option is, of course, simpy use `Y` for motion correction. Note the following line will not create a copy of `Y`, instead it simply bind the name `varr_mc` to `Y`.

In [ ]:
varr_mc = Y

### estimate shifts

The idea behind `estimate_shift_fft` is simple: for each frame it calculates a two-dimensional [cross-correlation](https://en.wikipedia.org/wiki/Cross-correlation) between that frame and a template using [fft](https://en.wikipedia.org/wiki/Fast_Fourier_transform). The argument `on` determine what we should use as template, it can be the first frame `on='first'`, the last frame `on='last'`, the mean frame `on='mean'`, or `on='perframe'`, meaning every frame will be registered to the immediate previous frame instead of a common template. After we have obtained the [cross-correlogram](https://en.wikipedia.org/wiki/Correlogram) for each frame, in theory we can simply look at the place/shift that has maximum correlation and use the inverse of that as `shifts`. However, due to some mysterious property of miniscope data, the maximum of the correlogram are usually biased towards weird places, most likely somewhere close to origin ((0, 0) shifts), and it's more reliable to use the [center_of_mass](https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.ndimage.measurements.center_of_mass.html) of the correlogram rather than simply [argmax](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.argmax.html). There is yet another gotcha -- there are usually other local maximas that will bias `center_of_mass` and it's better to threshold those out before we take the `center_of_mass`. The argument `pct_thres` controls that -- it will set any value below that percentile to 0 in the cross-correlogram. Pragmatically `pct_thres=99.9` works well at taking out all the weird artifacts. The results from `estimate_shift_fft` are saved in a two dimensional `DataArray` called `res`, with three different `variable`s: the shifts along `'height'`, the shifts along `'width'`, and the actuall value of maximum correlation `corr`. And we have these three `variable`s across all `frame`s (hence the two dimensions). We can thus pull out `shifts` and `corr`s from `res`.

In [ ]:
%%time
res = estimate_shift_fft(varr_mc, on='mean', pct_thres=99.9)
if in_memory:
    with ProgressBar():
        res = res.compute()
shifts = res.sel(variable=['height', 'width'])
corr = res.sel(variable='corr')

### masking and interpolation

Sometimes there are frames in the video that has a horrible shift and is simply way off the common field-of-view. For these frames it is not a good idea to shift them based on cross-correlation. Instead they should be treated as data loss and discarded/interpolated. `mask_shifts` tries to address this by simply look at the maximum correlations that is saved during `estimate_shift_fft`. The idea is if some frame has a way lower cross-correlation to template comparing to other frames, they are likely to be very bad frames. `mask_shifts` takes four required arguments: `varr_fft` which is the fft-transformed video arrays. `corr` which are the values of maximum correlations. `shifts` which are the shifts in both directions. All three of these comes from `estimate_shift_fft`. The fourth one `z_thres` gets to decide who is the "bad" frame -- the `corr`s will be z-scored, then any frame whose z-transformed `corr` is lower than `z_thres` would be treated as "bad" frames and interpolated. Another optional argument is `perframe`, it's a boolean value specifying whether `estimate_shift_fft` was carried out per-frame. If `perframe=True`, in addition to taking out "bad" frames, it will also recalculate the shifts of the two most adjacent "good" frames based on each other, that is, ignoring the "bad" frames who just got taken out. Obviously only set `perframe=True` if `on='perframe'` for `estimate_shift_fft`. The result of `mask_shifts` is an updated version of shifts `shifts_ma`, and a boolean `mask` specifying which are the "bad" frames.

In [ ]:
%%time
shifts_ma, mask = mask_shifts(varr_fft, corr, shifts, z_thres=-1.5)

After `mask_shifts`, we use the returned `mask` to actually interpolate the bad frames. The interpolation are simply a mean frame across two most immediate adjacent "good frames".

In [ ]:
%%time
varr_mc = interpolate_frame(varr_mc.compute(), mask)

### determine shifts

#### take cumulative sum if `on='perframe'` when estimating shifts

When we use `on='perframe'` for `estimate_shift_fft`, the shifts was calculated for each frame relative to the previous frame. Thus the real shift relative to the first frame should be a cumulative sum of `shifts`. Run the following cell only if you `on='perframe'` for `estimate_shift_fft`

In [ ]:
%%time
shifts_final = shifts.cumsum('frame')
shifts_final = np.around(shifts_final.fillna(0)).astype(int)

#### use raw shifts otherwise

If `on is not 'perframe'` for `estimate_shift_fft`, simply run the following line to take a round of `shifts` and convert them to integer datatype.

In [ ]:
shifts_final = np.around(shifts.fillna(0)).astype(int)

### apply shifts

Finally we apply the `shifts_final` we got to the movie we want, which is `Y` (not necessarily `varr_mc`).

In [ ]:
Y_mc = apply_shifts(Y, shifts_final)
if in_memory:
    with ProgressBar():
        Y_mc = Y_mc.persist()

### visualization of shifts

Here we visualize both the `shifts` and `shifts_final` as fluctuating curve along `frame`s. This is the first time we explicitly use the package [holoviews](http://holoviews.org) which is a really nice package for visualizing data, and I highly recommend reading through their tutorial to get familiar with the syntax.

In [ ]:
%%output size=100
%%opts Curve [width=500, tools=['hover']]
hv.NdOverlay(dict(width=hv.Curve(shifts.sel(variable='width')), height=hv.Curve(shifts.sel(variable='height'))))\
+ hv.NdOverlay(dict(width=hv.Curve(shifts_final.sel(variable='width')), height=hv.Curve(shifts_final.sel(variable='height'))))

### visualization of motion-correction

Here again we visualize the movies with `VArrayViewer`. The optional argument `framerate` only control how the frame slider behave, not how the data is handled. Note that `rename` here is necessary since `Y_mc` come from `Y` and share the same name. If we don't give them different names there is gonna be a `MergeError`.

In [ ]:
%%output size=100 fps=5
%%opts Image (cmap='Viridis')
vaviewer = VArrayViewer([Y.rename('Y'), Y_mc.rename('Y_mc')], framerate=5)
display(vaviewer.widgets)
vaviewer.show().redim.range(Y=(0,0.2), Y_mc=(0, 0.2))

### save result as DataSet

Finally we save the pre-processed and motion-corrected data to hard drive as before.

In [ ]:
%%time
save_variable(Y_mc.rename('Y'), dpath, 'minian', meta_dict=meta_dict)

After this it is a perfect time to restart the kernel to free up some memory, since we will only work with `Y` from now on and do not need any of those itermediate huge videos anymore.

## initialization

The whole initialization section is adapted from the [MIN1PIPE](https://github.com/JinghaoLu/MIN1PIPE) package. See their [paper](https://www.cell.com/cell-reports/fulltext/S2211-1247(18)30826-X) for full details about the theories. Here we only give minimal amount of information so that we can go through the parameters.

The first thing we want to do is open up the `minian.nc` dataset we have saved to hard drive. The [open_dataset](http://xarray.pydata.org/en/stable/generated/xarray.open_dataset.html) function itself is lazy and should take less than a second. After that we want to explicitly load up `Y` into memory to speed things up later.

In [ ]:
%%time
minian = xr.open_dataset(os.path.join(dpath, 'minian.nc'), autoclose=True)
Y = minian['Y'].load()

First step is to initialize the **seeds**. The idea is that we select some subset of frames, compute a max projection of those frames, then find the local maximums of that max projection. We keep repeating this process and putting together all the local maximums we get along the way, until we get an overly-complete set of local maximums/bright-spots, which are the potential location of cells and we call them **seeds**. The assumption here is that the center of cells have to be brighter than it's surroundings on some, but not necessarily all, frames. The first and only required argument `seeds_init` takes is `varr` -- the video array we want to process. There are four additional arguments controlling how we subset the frames: `wnd_size` controls the window size of each chunk, *i.e* the number of frames in each chunk; `method` can be either `'rolling'` or `'random'`. For `method='rolling'`, the moving window will roll along `frame`, whereas for `method='random'`, chunks with `wnd_size` number of frames will be randomly pick; `stp_size` is only used if `method='rolling'`, it is the step-size of the rolling windows, in other words the distance between the **center** of each rolling window. For example, if `wnd_size=100` and `stp_size=200`, the windows will be like: **(0, 50)**, **(150, 250)**, **(350, 450)** *etc.* (Obviously that was a bad choice since you probably want the windows to overlap otherwise you will miss cells); `nchunk` is only used if `method='random'`, It is the number of random chunks we will draw. The default values of `seeds_init` usually works fairly well.

In [ ]:
%%time
seeds = seeds_init(Y, wnd_size=500, method='rolling', stp_size=200, nchunk=100)

Now come the first step of cleaning the seeds -- by Gaussian Mixture Model. The idea is that for each seed we have, we calculate the difference between the peak and valley of the fluorecence signal corresponding to the seed pixel. If we put those peak-to-valley values for all the seeds together and look at the histogram of those values, they are supposed to be composed of two Gaussian distribution. We then fit a Gaussian Mixture Model of two components to those peak-to-valley values, and ask the model to predict for each value which Gaussian distribution it is likely to come from. We then keep only the seeds whose peak-to-valley value come from the Gaussian distribution with higher mean, that is, relatively large among all seeds. Thus the assumption here is that of all the "bright" seeds we have pulled out from the video, some of them barely changed their fluoresence across the whole recording session, while others changed a lot. Those who changes are cells whereas those who don't are weird random bright-spots. It can be speculated that if a cell was firing all the time during recording, they might be filtered out during this step. In that case you might want to skip this step. `gmm_refine` takes in the video array `varr`, the `seeds` to be refined, and an optional argument `q` controlling how we define "peak" and "valley". By default `q=(0.1, 99.9)` meaning we use **0.1** percentile as the "valley" and **99.9** percentile as the "peak". Usually it is fine to leave them as-is, but feel free to change them if you believe they are messing things up.

In [ ]:
%%time
seeds_gmm = gmm_refine(varr=Y, seeds=seeds, q=(0.1, 99.9))

`pnr_refine` stands for "peak-to-noise ratio" refine. The "peak" and "noise" here are defined differently from before. First we seperate/filter the temporal signal for each seed based on frequency -- the signals composed of the lower half of the frequency are regarded as **real** signals, while the higher half of the frequency are presumably **noise** ("half" being relative to [Nyquist frequency](https://en.wikipedia.org/wiki/Nyquist_frequency)). Then we take the peak-to-valley value (really just **max** minus **min**, or, [np.ptp](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.ptp.html)) for both the **real** signal and **noise** signal. Then finally, "peak-to-noise ratio" is simply the ratio between the `np.ptp` values of **real** and **noise** signal. So, the critical assumption here is that real activities are lower frequency while noise are higher frequency, and they seperate at approximately half the Nyquist frequency, or, one-fourth of the sampling frequency of the video, and we don't want those "seeds" whose **real** signals are buried in **noise**. If these assumptions does not suit your case, for example, if you have a really low sampling rate, or if your video are unavoidably noisy, consider skipping this step. As always, `pnr_refine` takes in `varr` and `seeds` as first two arguments, then a `thres`, where seeds whose "peak-to-noise ratio" fall below this threshold will be discarded. Pragmatically `thres=1.5` works fine.

In [ ]:
%%time
seeds_pnr = pnr_refine(varr=Y, seeds=seeds_gmm, thres=1.5)

`intensity_refine` threshold the seeds based purely on the intensity, and here is how the threshold is found: We first calculate a max projection across all frames, literally `varr.max('frame')`. Then, we bin the max projection so that each bin has [around](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.around.html) 10 pixels. We then use those bins to generate a [histogram](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.histogram.html), and find the bin that has the highest frequency/density. Then the threshold is simply twice the intensity that correspond to the highest bin. `intensity_refine` only takes `varr` and `seeds` with no additional arguments. It can, however, potentially run into problems: In case you have a field-of-view that is bright overall, the threshold might simply be out of the range of the intensities. In that case `intensity_refine` simply do nothing (and it will tell you if that happens). Pragamatically that rarely happens and `intensity_refine` usually does not take away real cells. However if you found this step too arbitrary you should be able to safely skip this.

In [ ]:
%%time
seeds_int = intensity_refine(varr=Y, seeds=seeds_pnr)

`ks_refine` refine the seeds using [Kolmogorov-Smirnov test](https://en.wikipedia.org/wiki/Kolmogorov–Smirnov_test). The idea is simple: for each seeds, their fluorescence level on all frames should be somewhat [bimodal](https://en.wikipedia.org/wiki/Multimodal_distribution), with a large normal distribution representing silence/little activities, and maybe a tiny peak representing when the seed/cell is firing. Thus we can carry out KS test for all seeds against the normal distribution, and keep only the seeds where the null hypothesis (that the fluoresence is simply a normal distribution) is rejected and is defaulted to **0.05**. `ks_refine` takes in `varr` and `seeds` as first two arguments, then a `sig` which is the significance level at which the null hypothesis is rejected. Personally I found this step tend to take away real cells when the video is very short (for example, the one that comes with this package under `"./demo_movies"`), probably because the number of "active" frames are too small comparing to the majority of the frames. So feel free to skip this step if you encounter the same situation.

In [ ]:
%%time
seeds_ks = ks_refine(varr=Y, seeds=seeds_int, sig=0.05)

Potentially we could pick out multiple seeds that is actually within one cell, and we want to avoid that as much as possible. `seeds_merge` try to merge seeds together that potentially come from the same cell, based on spatial distance and temporal correlation. Specifically, `thres_dist` is the threshold for euclidean distance between pair of seeds, in pixels; `thres_corr` is the threshold for pearson correlation between pair of seeds. Any pair of seeds that are within `thres_dist` **and** has a correlation higher than `thres_corr` will be merged together, where "merge" meaning only the seed with maximum internsity in the max projection of the video will be kept. Thus `thres_dist` should be the expected size of the cells and `thres_corr` should be relatively high to avoid over-merging.

In [ ]:
%%time
seeds_mrg = seeds_merge(varr=Y, seeds=seeds_ks, thres_dist=5, thres_corr=0.6)

Up till now, the seeds we have are only one-pixel dots. In order to kick start CNMF we need something more like the spatial footprint `A` and temporal activities `C` of real cells. Thus we need to `initilalize` `A` and `C` from the `seeds_mrg` we have. Here is how it is done:For each seed, the temporal activities of surrounding pixels can be put into a matrix `sur` with dimension `frame` and `spatial`, where `spatial` is the flattened representation of pixel location, or, a [stack](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.stack.html)ed version of `height` and `width`. Let `sd` denote the temporal activity of the seed, we use the dot product of the temporal activities between the seed and the surrounding pixels `sur.dot(sd)` normalized by the [Euclidean norm](https://en.wikipedia.org/wiki/Norm_(mathematics)) of `sur`: [np.linalg.norm(sur)](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.linalg.norm.html), as well as `np.linalg.norm(sd)`, as the spatial footprint `a`. In the form of equation: $$\mathbf{a} = \frac{\mathbf{sur} \cdot \mathbf{sd}}{\Vert{\mathbf{sur}}\Vert \Vert{\mathbf{sd}}\Vert}$$ We can think of this as a normalized measurement of how similar the temporal activities of each surrounding pixel is comparing to those of the seed. Then we use the dot product `sur.dot(a)` as the temporal trace for the seed, which is conceptually an "averaged" activities of all the pixels surrounding the seed weighted by `a`. After that we simply [concatenate](http://xarray.pydata.org/en/stable/generated/xarray.concat.html) the spatial footprints and temporal traces from different seeds together along a new dimension `unit_id` to get our initial `A` and `C` matrix. Finally we need two more terms: `b` and `f` representing the spatial footprint and temporal dynamic of the **background**, respectively. We find those by simply taking the dot product `A.dot(C)`, which is the activities from our initial units, and subtract that from the original movie `varr`. We use the mean of that remainder across `frame` as `b`, and the mean across `height` and `width` as `f`. `initilaize` takes in `varr` and `seeds` as the first two arguments. Two additional arguments determine how we find the `sur`roundings when calculating `a`. Firstly we certainly do not want to consider every other pixels in the movie as `sur` to a seed, and we just need a hard window where any pixels outside that window are ignored during the calculation of `a`. `wnd` is the half-size of that window. Secondly, not every pixels within the range of `wnd` are equally possible to be part of the cell represented by the seed. We thus put another correlation threshold -- pixels whose temporal activities has a pearson correlation with the seed higher than `thres_corr` will be considered as `sur`.

In [ ]:
%%time
A, C, b, f = initialize(varr=Y, seeds=seeds_mrg, thres_corr=0.8, wnd=10)

Finally we visualize the result of our initialization by plotting a projection of the spatial matrix `A` as well as a raster of the temporal matrix `C`

In [ ]:
opts = dict(plot=dict(height=300, width=300))
regrid(hv.Image(A.sum('unit_id'), kdims=['width', 'height'])).opts(**opts) + regrid(hv.Image(C, kdims=['frame', 'unit_id'])).opts(**opts)

Then we save the results in the dataset. Note here we change the name of a dimension by doing `rename(unit_id='unit_id_init')`, this is a precaution, since the dimension `unit_id` will likely to change after next section **CNMF** (most likely units will be merged), and there will be conflicts then if we save other variables with dimension `unit_id` that has different coordinates.

In [ ]:
%%time
save_variable(A.rename('A_init').rename(unit_id='unit_id_init'), dpath, 'minian', meta_dict=meta_dict)
save_variable(C.rename('C_init').rename(unit_id='unit_id_init'), dpath, 'minian', meta_dict=meta_dict)
save_variable(b.rename('b_init'), dpath, 'minian', meta_dict=meta_dict)
save_variable(f.rename('f_init'), dpath, 'minian', meta_dict=meta_dict)

## CNMF

This section assume you already have good knowledge about using CNMF as a method of extracting neural activities from the movie. If not, please read [the paper](https://www.sciencedirect.com/science/article/pii/S0896627315010843) before proceeding.

As a quick reminder and check, here is the essential idea of CNMF: We believe our movie `Y`, with dimension `height`, `width` and `frame`, can be written in (and thus break down as) the following equation: $$\mathbf{Y} = \mathbf{A} \cdot \mathbf{C} + \mathbf{b} \cdot \mathbf{f} + \epsilon$$ where `A` is the spatial footprint of each unit, with dimension `height`, `width` and `unit_id`, and `C` is the temporal activities of each unit, with dimension `unit_id` and `frame`. `b` and `f` are the spatial footpring and temporal activities of some background, and $\epsilon$ is the noise. Note that strictly speaking, matrix multiplication are usually only defined for two dimensional matrices, but our `A` here has three dimensions. So in fact we are taking the [tensor product](https://en.wikipedia.org/wiki/Tensor_product) of `A` and `C`, reducing the dimension `unit_id`. This might seem to complicate things (comparing to just treat `height` and `width` as one flattened `spatial` dimension), but I believe it help you understand the essence of why the equation make sense: when you take a dot product of any two "matrices" on certain **dimension**, all that is happening is a **product** followed by a **sum** -- you take the product for all pairs of matching numbers coming from the two "matrices", where "match" is defined by their index along the said dimension, and then you take the sum of all those products along the dimension. Thus when we take the tensor product of `A` and `C`, we are actually multiplying all those numbers in dimension `height`, `width` and `frame`, matched by `unit_id`, and then take the sum. Conceptually, for each unit, we are weighting the spatial footprint (`height` and `width`) by the fluorecense of that unit on given `frame`, which is the **product**, and then we are overlaying all units together, which is the **sum**. With that, the equation above is trying to say that our movie is made up of a weighted sum of the spatial footprint and temporal activities of all units, plus some background and noise.

Now, there is another rule about `C` that separate it from background and noise, and save it from being just some random matrix that happen to fit well with the data `Y` without having any biological meaning, and that is the second essential idea of CNMF: each "row" of `C`, which is the temporal trace for each unit, should be described as an [autoregressive process](https://en.wikipedia.org/wiki/Autoregressive_model)(AR process), with a parameter `p` defining the **order** of the AR process: $$ c(t) = \sum_{i=0}^{p}\gamma_i c(t-i) + s(t) + \epsilon$$ where $c(t)$ is the calcium concentration at time (`frame`) $t$, $s(t)$ is spike/firing rate at time $t$ (what we actually care), and $\epsilon$ is again some noise. Basicly this equation is trying to say that at any given time $t$, the calcium concentration at that moment $c(t)$ depends on the spike at that moment $s(t)$, as well as its own history up to `p` time-step back $c(t-i)$, scaled by some parameters $\gamma_i$s, plus some noise $\epsilon$. Another intuition of this equation come from looking at different `p`s: when `p=0`, calcium concentration is an exact copy of the spiking activities, which is probably not true; when `p=1`, calcium concentration is an exponential decay with instant rise triggered by the spikes; when `p=2`, calcium concentration is an exponential decay that has some rising time; when `p>2`, more convoluted waveforms start to emerge. (all of these with some noise, of course)

With that, all CNMF is doing is trying to find the `A` and `C` (along with `b` and `f`) that best describe `Y`. There are two more important practical concerns: Firstly we cannot solve this problem in one shot, we need to iteratively and separately update `A` and `C` to approach the true solution, and we need something to start with (that is what **initilization** section is about). Surprisingly often times 2 iterative steps seem to give us good enough results, but you can always add more iterations (and you should be ablt to easily do that after reading the comments); Secondly, by intuition you may define "best describe `Y`" as the results that minimize the noise $\epsilon$ (or residule, if you will). However we have to control for the [sparsity](https://en.wikipedia.org/wiki/Sparse_matrix) of our results as well, since we do not want every little random pixel that happen to correlate with a cell to be counted as part of the spatial footprint of the cell (non-sparse `A`), nor do we want a tiny spike at every frame trying to explain every noisy peak we observe (non-sparse `C`). Thus the balance between fidelity (minimizing error) and sparsity (minimizing non-zero entries) is an important idea for both the spatial and temporal update.

### loading data

First we load in our data from previous steps. The `'unit_id'` renamming lines are just for the precaution metioned at the end of **initialization** section.

In [ ]:
%%time
chk = chunks.copy()
chk['unit_id_init'] = chk.pop('unit_id')
minian = xr.open_dataset(os.path.join(dpath, 'minian.nc'), chunks=chk)
Y = minian['Y']
A_init = minian['A_init'].rename(unit_id_init='unit_id')
C_init = minian['C_init'].rename(unit_id_init='unit_id')
b_init = minian['b_init']
f_init = minian['f_init']

### estimate spatial noise

Since we can not just ask the computer to minimize the noise as much as possible, we have to get a sense of how much noise is expected before balancing sparsity and error during the minimizing process. `get_noise_fft` will calculate the noise power for all pixels using fft. Essentially we compute an fft-transform for every pixel indepedently, which left us with a [power spectral density](https://en.wikipedia.org/wiki/Spectral_density) with dimension `spatial` and `freq`, where `spatial` is flattened `height` and `width`, and `freq` is the dimension representing different frequency component of the signal. The power spectrum is the second returnned value of `get_noise_fft`, which is assigned to `psd` in the following code. We then need to define noise with the spectrum. By default `noise_range=(0.25, 0.5)`. Note that the number here is relative to the sampling frequency. So **0.5** actually represent Nyquist frequency and is the highest you can go as far as fft concerns. Thus **(0.25, 0.5)** is the higher frequency half of the signal, which is consistent with our assumption during **initialization** section. After choosing `noise_range`, we have to decide how to collapse across different `freq` to get a single number of noise power for each pixel. Three `noise_method`s are availabe: `noise_method='mean'` and `noise_method='median'` will use the mean and median across all `freq` as the estimation of noise for each pixel. `noise_method='logmexp'`is a bit more complicated -- the equation goes like: $sn = \exp( \operatorname{\mathbb{E}}[\log psd] )$ where $\exp$ is the [exponential function](Exponential_function), $\operatorname{\mathbb{E}}$ is the [expectation operator](https://en.wikipedia.org/wiki/Expected_value) (mean), $\log$ is [natural logarithm](https://en.wikipedia.org/wiki/Natural_logarithm), $psd$ is the spectral density of noise for any pixel and $sn$ is the resulting estimation of noise power. It is recommended to keep `noise_method='logmexp'` since this is the default behavior of the [CaImAn](https://github.com/flatironinstitute/CaImAn) package.

In [ ]:
%%time
sn_spatial, psd = get_noise_fft(varr=Y, noise_range=(0.25, 0.5), noise_method='logmexp')

After this we plot the power spectrum `psd`. Note that the range of `psd` are usually wild and it is usually helpful to limit the range of our color with [`redim.range()`](http://holoviews.org/user_guide/Annotating_Data.html)

In [ ]:
%%opts Image [height=300, width=800, colorbar=True] (cmap='Viridis')
psd_flt = psd.stack(spatial=['height', 'width'])
hv_psd = hv.Image(psd_flt.assign_coords(spatial=range(psd_flt.sizes['spatial'])).rename('psd'), kdims=['spatial', 'freq'])
regrid(hv_psd).redim.range(psd=(0, 1e-4))

### randomly select units for parameter exploring

We will soon do some parameter exploring before actually commit to a spatial update, since you do not want to do a 10-minutes' worth step only to find out you started with some rediculous value for certain parameter. Obviously we only want to run the parameter exploring on very small subset of data. So here we randomly select 10 units from `A_init.coords['unit_id']` with the help of [`np.random.choice`](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.choice.html)

In [ ]:
units = np.random.choice(A_init.coords['unit_id'], 10)

### test parameters for spatial update

Here we actually do the parameter exploring. It seem to be the most complicated chunk of code so far, but it is very important to read through it, since we will see similar things later and it is a very powerful piece of code that can help you visualize a lot of stuff. The basic idea is we run `update_spatial` on a small subset (`units`) of data using different parameters within a `for` loop, and after that visualize the result using `holoviews`. We will go line by line: first two lines create `opts_A` and `opts_C`, which is the plotting options for plotting `A` and `C`. If you haven't got a chance to check out [the tutorial](http://holoviews.org/user_guide/Customizing_Plots.html) on plotting options from `holoviews`, it is time. Then we create a `list` of values we want to try out. Since we are interested in the parameter `sparse_penal`ty, we create a `sprs_ls`. We then create an empty `dict`ionary to store the resulting `A`s, thus we call it `A_dict`. Then there is the `for` loop iterating through `sprs_ls`, with one of the values from the list as `cur_sprs` during each iteration. Within the loop, we run `update_spatial`. The exact arguments `update_spatial` needs will be discussed later, but basicly we just have to feed it a bunch of data, including the movie `Y`, initial values of `A_init`, `b_init`, `C_init`, and `f_init`, the spatial noise we just estimated `sn_spatial`. Then we feed in the parameter we are playing with `sparse_penal=cur_sprs`. Note that we subset our data here by `sel`ecting the ten `units` we just randomly chose. Then we created [`hv.Image`](http://holoviews.org/reference/elements/bokeh/Image.html) from the resulting `cur_A` and `cur_C`. Again if you haven't check out [the tutorial](http://holoviews.org/getting_started/Introduction.html) on how to create basic elements, please do. After that we combine the two image by simply `+` them together, and ask them to ocuupy only one column `.cols(1)`, and store this whole layout into `A_dict`, with the keys being the `cur_sprs` corresponding to the result of this iteration. After all the loops are done, we generate a [`hv.HoloMap`](http://holoviews.org/reference/containers/bokeh/HoloMap.html) from `A_dict` so that we can visualize all the results in an animated way.

In [ ]:
opts_A = dict(plot=dict(height=480, width=752), style=dict(cmap='Viridis'))
opts_C = dict(plot=dict(height=480, width=1000), style=dict(cmap='Viridis'))
sprs_ls = [5e-6, 5e-3, 0.5, 5]
A_dict = dict()
for cur_sprs in sprs_ls:
    cur_A, cur_b, cur_C, cur_f = update_spatial(
        Y, A_init.sel(unit_id=units),
        b_init, C_init.sel(unit_id=units), f_init, sn_spatial, sparse_penal=cur_sprs)
    hv_cur_A = hv.Image(cur_A.sum('unit_id'), kdims=['width', 'height']).opts(**opts_A)
    hv_cur_C = hv.Image(cur_C, kdims=['frame', 'unit_id']).opts(**opts_C)
    A_dict[cur_sprs] = (hv_cur_A + hv_cur_C).cols(1)
hv_res = hv.HoloMap(A_dict, kdims=['sparse_penalty'])

Finally, we actually plot out `hv_res`. What you should expect here will be explained later along with what `sparse_penal` actually do.

In [ ]:
%%opts Image [colorbar=True] {+axiswise}
hv_res.collate()

### first spatial update

Finally here is the idea behind `update_spatial`. To reiterate, the big picture is that given the data `Y` and `C` from previous update (which is `C_init` for now), we want to find the `A` so that 1. the **error** `Y - A.dot(C, 'unit_id')` is as small as possible, and 2. the [**l1-norm**](http://mathworld.wolfram.com/L1-Norm.html) of `A` is as small as possible. Here the **l1-norm** is a proxy to control for the sparsity of `A`. Ideally to promote sparsity we want to control for the number of non-zero entry in `A`, which is [l0-norm](https://en.wikipedia.org/wiki/Lp_space#When_p_=_0). However optimizing for l0-norm is usually considered [computationally hard to do](https://stats.stackexchange.com/questions/269298/why-do-we-only-see-l-1-and-l-2-regularization-but-not-other-norms), and it is usually good enough to use **l1-norm** instead as a proxy.

Now, in theory we want to update every entry in  `A` iteratively with the above two goals in mind. However updating that amount of numbers in `A` is still computationally very demanding, and it is much better if we can breakdown the big problem into smaller chunks so that we can parallelize them. **CNMF** is all about solving the issues caused by overlapping neurons, so it is best to keep the dependency along `unit_id` and update those entries together. However, it is totally fine to treat each pixel as independent and update different pixels separately (in parallel). Thus, our new "smaller" problem is: for each pixel, find the corresponding pixel in `A` across all `unit_id` that gives us smallest **l1-norm** as well as smallest **error** when multiplied by `C`. In equation, this is:

$$\begin{equation*}
\begin{aligned}
& \underset{A_{ij}}{\text{minimize}}
& & \left \lVert Y_{ij} - A_{ij} \cdot C \right \rVert ^2 + \alpha \left \lvert A_{ij} \right \rvert \\
& \text{subject to}
& & A_{ij} \geq 0 
\end{aligned}
\end{equation*}$$

where we use $A_{ij}$ to represent one pixel in `A` like `A.sel(height=i, width=j)`, which will only have one dimension `unit_id` left. Similarly $Y_{ij}$ is the corresponding pixel in `Y` which only have one dimension `frame` left. Thus $\left \lVert Y_{ij} - A_{ij} \cdot C \right \rVert ^2$ is our **error** term and $\left \lvert A_{ij} \right \rvert$ is our **l1-norm**, and we are finally putting them together as one goal, with an $\alpha$ controlling the balance between them. So the higher the $\alpha$, the more contribution the **l1-norm** term make to the common goal (target function), the more penalty/emphasis you are putting on sparsity, and as a result the more sparse `A` would become. The determination of the exact value of $\alpha$ is rather complicated, but the parameter we have for `update_spatial` is relative, where `alpha=1` correspond to the default behavior of **CaImAn** package, and is usually a good place to start testing. Pragmatically, relatively small value of `sparse_penal` has very little impact on the resulting `A`, but once you hit a large enough value, you will start to see units getting dimmer, sometimes completely disappear when you do [the parameter exploring](#test-parameters-for-spatial-update). You might think that is sparsity penalty in action, but from experience this is usually the case you want to **avoid**, since the sparse penalty we are imposing now not only limit the number of non-zero entries, but also their value (this is why **l1-norm** regulation is inferior to **l0-norm** regulation, by the way), and they usually mistakenly reduce the "intensity" of real cells in the resulting `A` to a very small value, which could potentially make the temporal update infeasible.

There is yet another parameter `dl_wnd` that is related to a practical consideration: recall that we are updating $A_{ij}$ for our "small" problem, which has the dimension `unit_id` and has `len(A.coords['unit_id'])` number of entries in them (that is, the number of units you have). This is computationally feasible, but still a lot, especially when you do this to all the pixels. One way to reduce the computational demands is to leave out certain units when updating certain pixels -- it does not make sense to consider a unit that is supposed to be at the top left corner of the field of view when we update a pixel in bottom right corner. In other words, for each pixel, we solve the "small" problem with only a subset of all the potential units, thus hugely increase the speed of `update_spatial`. This is where `A_init` comes into play (actually the only place it is used -- we do not need `A` at all for the update itself). We compute a morphological dilation that is used during [background removal](#background-removal) on `A_init`, unit by unit, with window size `dl_wnd`, and we use the result as a **masking matrix**, so that during the actual update of any given pixel, only the unit that has a non-zero value at the corresponding pixel in the **masking matrix** will be considered for updating. In other words, we are allowing each unit to expand from `A_init` up to a distance of `dl_wnd`, and kill off any possibility beyond that range, since they will not be considered for update with those pixels outside the **masking matrix**. We can again use the expected size of cell as `dl_wnd`, the rationale would be even, for some reason, we have only one non-zero pixel representing the center of certain unit in `A_init`, that unit can potentially expand to a full size cell, but anything beyond that would probably be either part of other cell or random noise. Obviously after `update_spatial`, if you find you units being cut-off abruptly and seem to be not able to expand enough, try increasing `dl_wnd`. If you do not like this feature and feel happy with the full scale computation, setting `dl_wnd=None` or `dl_wnd=0` will turn off the masking completely. 

`update_spatial` takes in the original data `Y`, the initial spatial footprint for units and background `A` and `b` respectively, the initial temporal trace for units and background `C` and `f` respectively, and the estimated noise on each pixel `sn`, in that order. Optional arguments are `sparse_penal` and `dl_wnd` which we already discussed, and yet another boolean argument specifying whether we should `update_background` -- notice that we have been completely sloppy with the background term during our discussion, mainly because the morphology-based [background removal](#backgroun-removal) works so well at cleaning the backgrounds, and it is usually fine to ignore the background completely and set `update_background=False`. If you set `update_background=True`, the initial `f` will be used to update `b` just like an additional unit, since there is nothing the differentiate unit from background during `update_spatial`.

In [ ]:
%%time
A_spatial, b_spatial, C_spatial, f_spatial = update_spatial(
    Y=Y, A=A_init, b=b_init, C=C_init, f=f_init, sn=sn_spatial,
    sparse_penal=0.1, dl_wnd=5, update_background=True)

After this we visualize the effect of `update_spatial` by plotting the input `A_init` along with the result `A_spatial` 

In [ ]:
%%opts Image [colorbar=True] (cmap='Viridis')
regrid(hv.Image(A_init.sum('unit_id'), kdims=['width', 'height'])).opts(plot=dict(height=480, width=752))\
+ regrid(hv.Image(A_spatial.sum('unit_id').rename('A_spatial'), kdims=['width', 'height'])).opts(plot=dict(height=480, width=752)).redim.range(A_spatial=(0, 0.5))

### test parameters for temporal update

Now we move on the temporal update. Here we use the same idea we have before to do some parameter exploring, only this time there is much more parameters to play with for temporal update, and we now have four `list`s of potential parameters: `p_ls`, `sprs_ls`, `add_ls`, and `noise_ls`. We use [`itertools.product`](https://docs.python.org/3.7/library/itertools.html#itertools.product) to iterate through all possible combinations of the potential values and save us from nested `for` loops.

In [ ]:
%%time
import itertools as itt
p_ls = [1]
sprs_ls = [0.5, 5]
add_ls = [5]
noise_ls = [0.25]
vis_dict = dict()
for cur_sprs, cur_p, cur_add, cur_noise in itt.product(sprs_ls, p_ls, add_ls, noise_ls):
    print("processing {}".format((cur_p, cur_sprs, cur_add, cur_noise)))
    YrA, cur_C, cur_S, cur_B, cur_C0, cur_sig, cur_g, = update_temporal(
        Y, A_spatial.isel(unit_id=slice(10, 20)), b_spatial, C_spatial.isel(unit_id=slice(10, 20)),
        f_spatial, sn_spatial, sparse_penal=cur_sprs, p=cur_p, use_spatial=False, use_smooth=True,
        add_lag = cur_add, noise_freq=cur_noise, chk=dict(frame=200, unit_id=20),
        cvx_sched="processes")
    vis_dict[(cur_p, cur_sprs, cur_add, cur_noise)] = visualize_temporal_update(
        YrA, cur_C, cur_S, cur_g, cur_sig)

In [ ]:
%%opts Curve [width=800] {+framewise}
hv_res = hv.HoloMap(vis_dict, kdims=['p', 'sparse_penalty', 'add_lag', 'noise_freq']).collate()
hv_res

### first temporal update

Here comes the idea for temporal update: just like spatial update, given `A`, our goal is to find the `C` that minimize both the **error** (`Y - A.dot(C, 'unit_id')`) and **l1-norm** of `C`. However there is an additional constraint: the trace of each unit in `C` must follow the autoregressive process. Due to this additional layer of complexity, everything becomes more computationaly expensive, and we have to do more to reduce the computation cost. The first observation is that `A` is usually much larger than `C` (you usually have more total pixels than `frame`s), and doing a dot product `A.dot(C, 'unit_id')` everytime you try a different number in `C` can easily go crazy. Thus we want to leverage this burden by converting our **error** term to something like $\mathbf{A}^{-1} \cdot \mathbf{Y} - \mathbf{C}$, where $\mathbf{A}^{-1}$ represent a matrix that can "undo" what `A` usually do to `C` -- instead of weighting the temporal activity of each unit by spatial footprint (converting a matrix with dimension `unit_id` and `frame` into one with `height`, `width` and `frame`), $\mathbf{A}^{-1}$ "extract" the temporal activities of each unit based on their spatial footprint (convert a matrix with dimension `height`, `width` and `frame` into one with dimension `unit_id` and `frame`), in other words, almost like an [inverse](https://en.wikipedia.org/wiki/Moore–Penrose_inverse) of `A`. This way, we only need to calculate $\mathbf{A}^{-1} \cdot \mathbf{Y}$ once and be done -- we can use that result everytime we update `C`. The calculation of $\mathbf{A}^{-1} \cdot \mathbf{Y}$ is rather complicated and not strictly mathematically accurate, but it provide a good approximation with huge computational benefit and is the default behavior of CaImAn. You can turn this off by supplying `use_spatial=True`, however that is usually too computationally demanding to do practically. We will assume `use_spatial=False` in the following discussion and call the $\mathbf{A}^{-1} \cdot \mathbf{Y}$ term `YrA` as in the code. The second observation we have about computational complexity is that we can no longer keep the `unit_id` dimension and chop up `frame` dimension for parallel processing (like what we did for spatial update, except we chopped up pixels that time), since we have to check whether each trace along the `frame` dimension follow the autoregressive process this time. So unfortunately we have to turn to the `unit_id` dimension to make our problem "smaller". Since we have a relatively good `A` now, it should be OK to update units that are not spatially overlapping independently. This idea should work if you have a relatively sparse distribution of cells. However if your field-of-view is packed with cells, like during an imaging of hippocampus, and we consider cells overlap with each other if ther share more than one pixels, we will likely ended up updating the whole `C` altogether anyway since every cell is transitively overlapping with every other cell. So we need to put an threshold on how we define "overlap", and that is what `jac_thres` for -- only the cells that share their area of spatial footprint more than this threshold (ranging from 0 to 1) will be considered "overlapping". (the "proportion of overlapping area" has a formal name [Jaccard index](https://en.wikipedia.org/wiki/Jaccard_index), hence the name `jac_thres`). Pragamatically `jac_thres=0.2` works for data that is very compact in cells.

We now turn to the "other layer of complexity" which is the autoregressive process. Recall that the temporal trace of each unit should be fitted by the following equation: $$c(t) = \sum_{i=0}^{p}\gamma_i c(t-i) + s(t) + \epsilon$$ The first thing we want to determine is `p`. As discussed before, `p=2` is a good choice if your calcium transients has an observable rising time. `p=1` might work better if the rising time is so fast comparing to your sampling rate that you want to ignore it. `p` more than 3 might give you issues of [over-fitting](https://en.wikipedia.org/wiki/Overfitting) and is generally not recomended unless you are certain that your calcium traces appear to have a more complicated waveform that cannot be accounted for by a rise and decay. Next, notice that we have a bunch of $\gamma_i$s unaccounted for (usually not that much denpending on the `p` you choose). Luckily we do not have to iteratively update those -- it turns out that the $\gamma_i$s of an autoregressive process has certain relationship with the [autocovariance](https://en.wikipedia.org/wiki/Autocovariance) functions of the signal with with different lags, which can be readily computed from `YrA`. For full derivation of those relationships please refer to the [original CNMF paper](https://www.sciencedirect.com/science/article/pii/S0896627315010843?via%3Dihub). Here we can just speculate that the parameters that affect how much a signal depends on its own history must have some relationship with the covariance of the signal when you shift itself with different lags. So $\gamma_i$s can be computed rather deterministicly. Say you choose `p=2` and thus you have two $\gamma_i$s to be estimated, you would need exactly two equations involving the autocovariance function up to 2 time-step lags to give you the two $\gamma_i$s. However you can keep adding in more equations, since the impact of $\gamma_i$s can extend infinitely back in time and should be refelected in autocovariance functions with all those additional lags. Then you have an [overdetermined system](https://en.wikipedia.org/wiki/Overdetermined_system) which, in an ideal world, would just give you the one true answer of the two $\gamma_i$s that satisfy all those equations. In reality nothing adds up perfectly and we solve those equations with [least squares](https://en.wikipedia.org/wiki/Least_squares). Thus it is important to choose an appropriate number of **additional** equations, which is what `add_lag` control. An `add_lag` that is too small like `add_lag=0` will leave everything to the first `p` number of equations and autocovariance functions, which might not be reliable. Pragmatically I found smaller value of `add_lag` tends to bias the $\gamma_i$s to give a much faster decay, whereas larger `add_lag` tend to give a longer decay. As a rule of thumb it is usually good to set `add_lag` to approximately the decay time of your signal (in frames). Once we determined the $\gamma_i$s, the calcium traces $c(t)$ and spikes $s(t)$ are essentially **one thing** -- given calcium traces and how they rise/decay in response to spikes, we can deduce where the spikes happens, and *vice versa*. We can express this determined relationship with a matrix $\mathbf{G}$ where $s(t) = \mathbf{G} \cdot c(t)$. In other words, $\mathbf{G}$ is the matrix that "undo" what $\gamma_i$s do to $s(t)$. With all these parameters sorted out, we finally come to the actual optimization problem:

$$\begin{equation*}
\begin{aligned}
& \underset{C_{i}}{\text{minimize}}
& & \left \lVert \mathbf{YrA}_{i} - \mathbf{C}_{i} \right \rVert ^2 + \alpha \left \lvert \mathbf{G}_{i} \cdot \mathbf{C}_{i} \right \rvert \\
& \text{subject to}
& & \mathbf{C}_{i} \geq 0, \; \mathbf{G}_{i} \cdot \mathbf{C}_{i} \geq 0 
\end{aligned}
\end{equation*}$$

just as during spatial update, we select some units $i$, and update their calcium dynamics $\mathbf{C}_i$ based on the **error** and the **l1-norm** of the **spikes** $\mathbf{G}_i \cdot \mathbf{C}_i$. Again it does not make sense to have negative calcium dynamics or spikes so that is our constraint on the problem. Again we need an $\alpha$ to balance our weight on fidelity and sparsity, which can be scaled up and down with `sparse_penal`, with `sparse_penal=1` equivalent to the default behavior of CaImAn. Again $\alpha$ should depend on the expected level of noise. Note that we cannot use `sn_spatial` since we need the noise for each unit, not each pixel. Thus we have to calculate it again. `update_temporal` do that for you -- you just have to tell it the `noise_freq`uency. Again **0.5** is the highest you can go and the default `noise_freq=0.25` means the higher frequency half of the signal will be considered noise. In addition to affecting the estimation of noise power, `noise_freq` affect another smoothing process: when we were estimating $\gamma_i$s using the autocovariance of the signal, it is usually helpful to run a filter on the signal to get rid of all the high freqeuency noise, especially when you don't have large `add_lag` and the estimation can be easily biased by random noise. `noise_freq` is the cut-off frequency of the low-pass filter that run on the temporal trace for each unit, and you can pass in `use_smooth` to control whether to do the filtering at all. Finally even with all these careful design on the problem, it is sometimes hard to approach the true solution. When that happens `update_temporal` will warn you about it by saying something like "problem solved sub-optimally". Usually a few of those warnings is OK, but if you see that a lot it means you either have some unreasonable parameters or you need more iterations to approach the real answer. You can use `max_iters` to control how many iterations to run for each small problem before the computer gave up and throw a warning. Note that increasing `max_iters` will significantly increase the computation time and will not help at all if the parameters are wrong, so try to use this only as a last resort.

`update_temporal` takes in `Y`, `A`, `b`, `C`, `f`, and `sn_spatial` (even if we won't need it by default), in that order. Optionally you can pass in `noise_freq`, `p`, `add_lag`, `jac_thres`, `use_spatial`, `sparse_penal`, `max_iters` and `use_smooth` as we have discussed. `update_temporal` returns much more than we expected -- in addition to `C_temporal` and `S_temporal`, which is the result we care about, it also give you `YrA`, and `g_temporal` which is the $\mathbf{G}$ matrix for each unit. Moreover there are `B_temporal`, `C0_temporal` and `sig_temporal`, which is the final layer of complexity: when we update the temporal trace, there might be a global baseline calcium concentration, which is modeled by $b$ and returned in `B_temporal`. There might also be a spike that happened right before recording starts and is still decaying in the first few seconds, so we model this with an initial calcium concentration $c_0$ that follows the same decaying pattern defined by $\gamma_i$s, and is returned in `C0_temporal`. Both $b$ and $c_0$ is a single number that gets updated along with the calcium dynamics for each unit. Finally there is `sig_temporal` which is the combination of all the signals, that is: `C_temporal + C0_temporal + B_temporal`

In [ ]:
%%time
YrA, C_temporal, S_temporal, B_temporal, C0_temporal, sig_temporal, g_temporal = update_temporal(
        Y, A_spatial,
        b_spatial, C_spatial, f_spatial, sn_spatial, jac_thres=0.2,
        noise_freq=0.45, sparse_penal=0.5, p=1, add_lag = 0, use_spatial=False)

Then we get to visualize the results. There are four traces in the top plot: "Raw Signal" is `YrA`, "Fitted Spikes" is `S_temporal`, "Fitted Calcium Trace" is `C_temporal` and "Fitted Signal" is `sig_temporal`. The latter two traces usually overlap with each other since `B_temporal` and `C0_temporal` are often times just **0**. Sadly due to large number of frames and the limitation of our browser, it is usually only possible to visualize 50 units at a time, hence `select(unit_id=slice(0, 50))`, nevertheless it can give us an idea about how things goes. Put in other numbers if you want to see other units. 

In [ ]:
%%opts Curve [width=1200] {+framewise}
visualize_temporal_update(YrA, C_temporal, S_temporal, g_temporal, sig_temporal).select(unit_id = slice(0, 50))

Then there is another raster plot of the temporal traces for all the units with both `C_temporal` and `S_temporal`.

In [ ]:
%%opts Image [colorbar=True] (cmap='Viridis')
hv_c = regrid(hv.Image(C_temporal.rename('c'), kdims=['frame', 'unit_id'])).opts(plot=dict(height=500, width=1000)).redim.range(c=(0, 1))
hv_s = regrid(hv.Image(S_temporal.rename('s'), kdims=['frame', 'unit_id'])).opts(plot=dict(height=500, width=1000)).redim.range(s=(0, 0.006))
(hv_c + hv_s).cols(1)

### merge units

One thing CNMF cannot do is to merge together units that belong to the same cell. Even though we tried something similar during [initialization](#initialization), we might miss some, and it is better to do it here again. The idea is straight-forward -- it is based purely on pearson correlation of temporal activities. Any units that share at least one pixel in spatial footprint are considered potential target for merging, and any of those units that has a pearson correlation of temporal activities higher than `thres_corr` will actually be merged. 

In [ ]:
%%time
A_mrg, sig_mrg = unit_merge(A_spatial, sig_temporal, thres_corr=0.8)

### second spatial update

Here comes the second iteration of spatial update. It is identical to [first spatial update](#first-spatial-update) except there are appending **it2**s after the variable names standing for "iteration 2", which is completely optional is you do not care about comparing results from different iterations. I hope it is very apparent now that if you want more iteration, just add more of this sections, as well as [the section below](#second-temporal-update).

In [ ]:
%%time
A_spatial_it2, b_spatial_it2, C_spatial_it2, f_spatial_it2 = update_spatial(
    Y, A_mrg, b_spatial, sig_mrg, f_spatial, sn_spatial, sparse_penal=0.1, dl_wnd=5)

In [ ]:
A_spatial_it2_norm = xr.apply_ufunc(normalize, A_spatial_it2, input_core_dims=[['height', 'width']], output_core_dims=[['height', 'width']], vectorize=True)

In [ ]:
regrid(hv.Image(A_spatial.sum('unit_id'), kdims=['width', 'height'])).opts(plot=dict(height=480, width=752))\
+ regrid(hv.Image(A_spatial_it2_norm.sum('unit_id'), kdims=['width', 'height'])).opts(plot=dict(height=480, width=752))

### second temporal update

This section is identical to the [first temporal update](#first-temporal-update) except for variable names.

In [ ]:
%%time
YrA, C_temporal_it2, S_temporal_it2, B_temporal_it2, C0_temporal_it2, sig_temporal_it2, g_temporal_it2 = update_temporal(
    Y, A_spatial_it2, b_spatial_it2, C_spatial_it2, f_spatial_it2, sn_spatial, jac_thres=0.2,
    noise_freq=0.45, sparse_penal=0.5, p=1, add_lag=0)

In [ ]:
%%opts Curve [width=1200] {+framewise}
visualize_temporal_update(
    YrA, C_temporal_it2, S_temporal_it2, g_temporal_it2, sig_temporal_it2).select(unit_id=slice(0, 50))

In [ ]:
%%opts Image [colorbar=True, tools=['hover']] (cmap='Viridis')
hv_c = regrid(hv.Image(C_temporal_it2.rename('c'), kdims=['frame', 'unit_id'])).opts(plot=dict(height=500, width=1000)).redim.range(c=(0, 1))
hv_s = regrid(hv.Image(S_temporal_it2.rename('s'), kdims=['frame', 'unit_id'])).opts(plot=dict(height=500, width=1000)).redim.range(s=(0, 0.006))
(hv_c + hv_s).cols(1)

### save results

And finally we save our results in the `minian` dataset. Note that you can save any other variables by calling `save_variable`. For example, you might want to consider using `sig_temporal`s instead of `C_temporal`s for your downstream analysis. Also you are not restricted to use the [netcdf](https://www.unidata.ucar.edu/software/netcdf/) format though it is recommended. [Explore the xarray documentation](http://xarray.pydata.org/en/stable/io.html) for all IO options, and moreover [numpy IO capabilities](https://docs.scipy.org/doc/numpy-1.15.0/reference/routines.io.html) since `xarray` is built on top of `numpy`.

In [ ]:
%%time
minian.close()
save_variable(A_spatial_it2.rename('A'), dpath, 'minian', meta_dict=meta_dict)
save_variable(C_temporal_it2.rename('C'), dpath, 'minian', meta_dict=meta_dict)
save_variable(S_temporal_it2.rename('S'), dpath, 'minian', meta_dict=meta_dict)
save_variable(g_temporal_it2.rename('g'), dpath, 'minian', meta_dict=meta_dict)
save_variable(b_spatial_it2.rename('b'), dpath, 'minian', meta_dict=meta_dict)
save_variable(f_spatial_it2.rename('f'), dpath, 'minian', meta_dict=meta_dict)

### visualization

Here we load up the data we just saved for visualization purpose.

In [ ]:
minian = xr.open_dataset(os.path.join(dpath, 'minian.nc'))

And here we call `generate_videos` to get a video that can help us quickly visualize the results. `generate_videos` takes in the dataset that contains all the needed variables, the full path to the output video file, and a `dict` specifying chunks for performance. The resulting video will have four parts - top left is the **Raw Video** `minian['org']`; Top right is the **Processed Video** `minian['Y']` (that is, after pre-processing and motion correction); Bottom left if the **Units** from CNMF `minian['A'].dot(minian['C'], 'unit_id')`; Bottom right is the **Residule**, that is **Raw Video** - **Units**.

In [ ]:
%%time
generate_videos(minian, os.path.join(dpath, "minian.mp4"), chk=dict(height=100, width=100, frame=1000))